<a href="https://colab.research.google.com/github/huijunam/CJ_AI_hackathon/blob/eunhy/addGIPHY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
os.chdir('/content/drive/MyDrive/CJ')

In [3]:
from IPython.display import clear_output

!pip install -r "/content/drive/MyDrive/CJ/requirements.txt"
clear_output()

In [4]:
from function import BERTClassifier, get_kobert_model, BERTSentenceTransform,  BERTDataset

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from kobert_tokenizer import KoBERTTokenizer
import gluonnlp as nlp
from transformers import BertModel
import re

In [6]:
chatbot_data_1 = pd.read_excel('/content/drive/MyDrive/CJ/한국어_단발성_대화_데이터셋.xlsx')
chatbot_data_2 = pd.read_excel('/content/drive/MyDrive/CJ/한국어_연속적_대화_데이터셋.xlsx')

In [7]:
chatbot_data_1= chatbot_data_1.iloc[:,:2]
chatbot_data_2 = chatbot_data_2.iloc[1:, :3]
chatbot_data_2 = chatbot_data_2.iloc[:,1:]

# 90 이하인 카테고리 추출
value_counts = chatbot_data_2['Unnamed: 2'].value_counts()
categories_to_remove = value_counts[value_counts <= 90].index

# 해당 카테고리를 포함하지 않는 행만 선택
chatbot_data_2 = chatbot_data_2[~chatbot_data_2['Unnamed: 2'].isin(categories_to_remove)]

chatbot_data_2 = chatbot_data_2.dropna(how='any')

#최종 데이터
df_data = pd.concat([chatbot_data_1,chatbot_data_2], ignore_index=True)

In [8]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

for cls in target_classes:
    df_data.loc[(df_data['Emotion'] == cls), 'Emotion'] = target_classes[cls]

data_list =[]
# ['sentence', 'class'] 형태로 변환
for q, label in zip(df_data['Sentence'], df_data['Emotion']):
    data =[]
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [9]:
#from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [10]:
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

In [13]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

In [14]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/CJ/base_model_koBERT_epoch3_.pt'))

<All keys matched successfully>

In [15]:
from konlpy.tag import Okt
okt = Okt()

In [124]:
s = pd.read_excel("/content/drive/MyDrive/CJ/stopwords.xlsx")
stop_words = set(list(s['stop_words']))

In [118]:
from keybert import KeyBERT
kw_model = KeyBERT(model)
clear_output()

In [130]:
# 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

def predict_and_keword(predict_sentence):
  new_data =[[predict_sentence, '0']]
  # print(new_data)

  new_dataset = BERTDataset(new_data, 0, 1, tokenizer, vocab ,max_len, True, False)
  new_dataloader = torch.utils.data.DataLoader(new_dataset, batch_size = batch_size, num_workers =5)

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(new_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    test_eval =[]
    for logit in out:
        if reverse[np.argmax(logit.detach().cpu().numpy())] != '중립':
            test_eval.append(reverse[np.argmax(logit.detach().cpu().numpy())])

    # 키워드 추출: 명사, 동사, 형용사 추출하기
    predict_sentence = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', predict_sentence)

    keywords = kw_model.extract_keywords(predict_sentence, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
    keyword = [x for x,y in keywords]
    keyword = ' '.join(keyword)
    key = [x for (x, y) in okt.pos(keyword, stem=True) if y in ['Noun']]

    key_v1 = [word for word in set(key) if not word in stop_words]

    return test_eval+key_v1

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [131]:
# 하이퍼 파라미터 설정
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
random_seed = 42

In [132]:
reverse =dict(map(reversed, target_classes.items()))

In [133]:
import json
import random
from urllib import parse, request
import re
from IPython.display import Image
import ipyplot

url = "http://api.giphy.com/v1/gifs/search"

In [134]:
sentence = input()
array = predict_and_keword(sentence)
print(array)
data_list = []
for item in array:
    params = parse.urlencode({
        "q": item,
        "api_key": "P9SzTTpADkMNu3XjLkzRyMxVGm3MyKDQ",
        "limit": "20"
        })

    try:
        with request.urlopen("".join((url, "?", params))) as response:
            data = json.loads(response.read())

        get_nums = set()

        while len(get_nums) < (len(data['data']))/2:
            get_nums.add(random.randint(0,len(data['data'])+1))

        for i in get_nums:
            #print(data['data'][i]['images']['downsized']['url'])
            data_list.append(data['data'][i]['images']['downsized']['url'])

    except:
        pass

get_nums = set()
while len(get_nums) != 15:
    get_nums.add(random.randint(0,len(data_list)-1))

data_gif = []
for i in get_nums:
    data_gif.append(data_list[i])


for idx, value in enumerate(data_gif):
    data_gif[idx] = re.sub('media+[0-9]', 'i', value)

ipyplot.plot_images(data_gif,img_width=200,show_url=False)

대단한 기억력이다 난 어제 몇시쯤 멀했는지도 기억이잘안나는데..!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/1 [00:00<?, ?it/s]

['놀람', '기억', '어제']
